# ODECO, Online Training Programme, Module 3, Assignment 1

## Online Training Session

### Introduction

The **European Data Portal** ([data.europa.eu](https://data.europa.eu)) has a user interface for searching at <https://data.europa.eu/data/datasets?locale=en&minScoring=0>.

However, if we want to build an application (software) reusing some of the datasets publicly available at the portal, we need to write some code accessing directly the APIs offered by data.europa.eu. At the [FAQ](https://data.europa.eu/en/faq) of the European Data portal we can find a compilation of accesible APIs (see question _"Which APIs are available and where can I find information about them?"_).

In this short tutorial/assignment we will show how to use the following APIs for searching:

- the **CKAN Restful API** (<https://data.europa.eu/api/hub/search/#tag/Ckan>): [CKAN API](http://docs.ckan.org/en/2.9/api/) also available in many other Open Data Catalogues deployed with [CKAN software](https://ckan.org/). CKAN is an open-source DMS (data management system) for powering data hubs and data portals. 
- the **Search Restful API** (<https://data.europa.eu/api/hub/search/#tag/Search>) : particular API developed for the European Data Portal.
- the **SPARQL end-point** (<https://data.europa.eu/data/sparql?locale=en>): standard **[SPARQL](https://www.w3.org/TR/sparql11-overview/)** end-point, also available in many other Open Data catalogues. SPARQL is a semantic query language for databases—able to retrieve and manipulate data stored in **[RDF](https://www.w3.org/TR/rdf11-concepts/)** format.

As the programming language to see that the use of the APIs is feasible, we will use **Python**.

### Let's start with CKAN Restful API

Let's start preparing a query to retrieve datasets from the European Data Portal. First, we will type a keyword. For instance, if we want to retrieve the datasets produced in the European [TRAFAIR](https://trafair.eu/) project, we could use the specific term `Trafair`.

In [ ]:
keyword = input('Type a keyword : ')

One initial possibility is to use the `package_search` function of the CKAN API (see <https://data.europa.eu/api/hub/search/#operation/ckanPackageSearch> and <https://docs.ckan.org/en/2.9/api/index.html#ckan.logic.action.get.package_search> for more details about the allowed parameters).

In [ ]:
edp_url = 'https://data.europa.eu/data/search'

search_request = edp_url + '/ckan/package_search?q=%22'+keyword +'%22&rows=10'

print('URL of the search request: ', search_request, '\n')

from urllib.request import urlopen

response = urlopen(search_request)

import json

jsonResponse = json.load(response)

print(jsonResponse)

As the output is JSON format and is not easy to read by humans, we should extract specific fields, e.g. the identifier (`id), with the following code:

In [ ]:
rows = jsonResponse["result"]["results"]
print(len(rows), ' results:\n')
for row in rows:
    print(row["id"])

### The Search Resful API as alternative

A second possibility is to use the Search Restful API of the portal (see <https://data.europa.eu/api/hub/search/#operation/searchGet> )

In [ ]:
search_request = edp_url + '/search?q=%22'+keyword +'%22&limit=10'

print('URL of the search request: ', search_request, '\n')

response = urlopen(search_request)

jsonResponse = json.load(response)

rows = jsonResponse["result"]["results"]
print(len(rows), ' results:\n')
for row in rows:
    print(row["id"])

### Use SPARQL for specific queries

If we want to make more specific queries filtering the values of specific properties, we can construct a [SPARQL query](http://www.w3.org/TR/sparql11-overview/) including a filter on the `dcat:keyword` property using this keyword and a [regular expression](https://en.wikibooks.org/wiki/SPARQL/Expressions_and_Functions#REGEX).

In [ ]:
query_text = """
           PREFIX dct:<http://purl.org/dc/terms/>
           PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
           PREFIX dcat: <http://www.w3.org/ns/dcat#>
           SELECT DISTINCT ?s WHERE { 
             ?s a dcat:Dataset .
             ?s dcat:keyword ?value . 
             FILTER regex(str(?value), '\\\\b"""+ keyword +"""\\\\b', 'i') .
           }"""

print(query_text)

We could run this query at the user interface of the SPARQL end-point at <https://data.europa.eu/data/sparql?locale=en>

But in our case, we will launch the query through our Python code:

In [ ]:
!pip install SPARQLWrapper

from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper('https://data.europa.eu/sparql')

sparql.setQuery(query_text)

sparql.setReturnFormat(JSON)

results = sparql.query().convert()


We need to process the results in JSON format as follows:

In [ ]:
print(len(results["results"]["bindings"]), ' results:\n')
for row in results["results"]["bindings"]:
    dataset = row["s"]["value"]
    print(dataset)

We can click on one of the previous links to see the metadata of a dataset in a human-readable format.
We can also retrieve the metadata in [Turtle format](https://www.w3.org/TR/turtle/) (a serialization format for RDF metadata) of one of these datasets (e.g. <https://data.europa.eu/88u/dataset/https-opendata-aragon-es-datos-catalogo-dataset-oai-zaguan-unizar-es-95726> ), and upload it in a graph:

In [ ]:

def get_file_name(url):
    """
    https://europeandataportal.eu/set/data/https-opendata-aragon-es-datos-catalogo-dataset-oai-zaguan-unizar-es-89319
    return https-opendata-aragon-es-datos-catalogo-dataset-oai-zaguan-unizar-es-89319
    """
    words = url.split('/')
    file_name = words[len(words)-1]
    return file_name

import rdflib

def download_metadata(url):
    """
    Parses the dataset with URL in the graph
    """
    id = get_file_name(url)
    ttl_url = 'https://data.europa.eu/data/api/datasets/'+ id + '.ttl?useNormalizedId=true&locale=en'
    print(ttl_url)
    try:
        graph = rdflib.Graph()
        graph.parse(ttl_url, format="turtle")
        return graph
    except Exception as err:
        print(f'Other error occurred: {err}')

url = input('Copy here one of the previous URLs: ')

graph = download_metadata(url)
v = graph.serialize(format='turtle')
print(v)

We can also retrieve the real data behind a dataset. 

For instance, this dataset (<https://data.europa.eu/88u/dataset/https-opendata-aragon-es-datos-catalogo-dataset-oai-zaguan-unizar-es-95726>) has an associated distribution in CSV format.

We can retrieve the URL of the CSV file making a SPARQL query on the metadata in Turtle format:

In [ ]:
url = "http://data.europa.eu/88u/dataset/https-opendata-aragon-es-datos-catalogo-dataset-oai-zaguan-unizar-es-95726"

graph = download_metadata(url)

def get_csv_urls(graph):
    query_text = """
                   PREFIX dct:<http://purl.org/dc/terms/>
                   PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
                   PREFIX dcat: <http://www.w3.org/ns/dcat#>
                   PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
                   SELECT  ?value  WHERE {
                        ?dataset dcat:distribution ?resource .
                        ?resource a dcat:Distribution .
                        ?resource dct:format <http://publications.europa.eu/resource/authority/file-type/CSV> .
                        ?resource dcat:accessURL ?value
                    }
                    """
    # print(query_text)
    qres =  graph.query(query_text)
    csv_files = []
    for row in qres:
        csv_files.append(row["value"])
    return csv_files

csv_urls = get_csv_urls(graph)
for row in csv_urls:
  print(row)


Last, we can visualize the content of the CSV file:

In [ ]:
import pandas as pd

for row in csv_urls:
  raw_sensor_data_2020_01 = pd.read_csv(row)
  print(raw_sensor_data_2020_01)

## Assignment

Now, you are going to search for datasets relevant to your own research projects.

You can make a copy of this Jupyter Notebooks in your own Google Drive accounts to be able to modify the variable called `final_query` in the cell below.

This variable contains a SPARQL query. Now you can make a more specific query filtering the datasets described with specific properties and values according to the [DCAT-AP metadata model](https://data.europa.eu/w21/c18bb81e-43fe-405f-9bad-bb58d30a3666). You can have a look at its [UML class model](https://github.com/SEMICeu/DCAT-AP/blob/2.1.0-draft/releases/2.1.0/dcat-ap_2.1.0.png).

For instance, you can try to retrieve datasets according to these two criteria:

- the datasets have been published by an organization close to your research lab (your University, municipality council, local government). The `dct:publisher` property and its contained `foaf:name` property will be relevant for this.
- the datasets hold at least one distribution in CSV format. The previous code to retrieve the URL of CSV data (see `get_csv_urls` function) will be relevant to see how to navigate through `Distributions`.

You can also try another query with these two other criteria:

- the title contains a specific term. The `dct:title` property will be relevant for this.
- the datasets hold at least one distribution in CSV format. The previous code to retrieve the URL of CSV data (see `get_csv_urls` function) will be relevant to see how to navigate through `Distributions`.

_Note: Just include two criteria in your queries. Probably, queries with more than two criteria will not be resolved quickly by the European Data portal SPARQL end-point within its time-out limit._

In [ ]:
def see_csv_data(dataset):
  graph = download_metadata(dataset)
  csv_urls = get_csv_urls(graph)
  for row in csv_urls:
    print(row)
    csv_file = pd.read_csv(row)
    print(csv_file)


def see_metadata(dataset):
  graph = download_metadata(dataset)
  v = graph.serialize(format='turtle')
  print(v)
  response = input('Do you want to see CSV data if available? [Y]es, [Any other key] for no ').upper()
  if response == 'Y':
    see_csv_data(dataset)

def search_datasets(url, sparql_query):
    sparql = SPARQLWrapper(url)
    sparql.setQuery(sparql_query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    print(len(results["results"]["bindings"]), ' results:\n')
    for row in results["results"]["bindings"]:
      dataset = row["s"]["value"]
      print(dataset)
      response = input('Do you want to see metadata? [Y]es, [N]o, [Q]uit ').upper()
      if response not in "YNQ" or len(response) != 1:
        print("I don't know how to do that")
        continue
      if response == 'Y':
        see_metadata(dataset)
      elif response == 'Q':
        break


edp_sparql = 'https://data.europa.eu/sparql'

final_query = """
           PREFIX dct:<http://purl.org/dc/terms/>
           PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
           PREFIX dcat: <http://www.w3.org/ns/dcat#>
           PREFIX foaf: <http://xmlns.com/foaf/0.1/>
           SELECT DISTINCT ?s WHERE { 
             ?s a dcat:Dataset .
             ?s dcat:keyword ?value . 
             FILTER regex(str(?value), '\\\\bTrafair\\\\b', 'i') .
           } LIMIT 100"""
print(final_query)

search_datasets(edp_sparql, final_query)